# INTRODUCTION
In the digital age, the volume of books available online has grown exponentially, making it increasingly difficult for readers to discover content that matches their interests. Recommendation systems have emerged as powerful tools to personalize user experiences and improve engagement across platforms. An Online Book Recommender System uses data about user preferences and item features to suggest books that users are likely to enjoy, based on their reading history or similarity to other users

## PROBLEM STATEMENT
With thousands of books being added to online platforms every day, users often face challenges in choosing what to read next. This leads to decision fatigue and can reduce user satisfaction and engagement. Without personalized recommendations, users may overlook books that align with their tastes, while content creators and publishers may struggle to reach their target audience. Therefore, there is a need for an intelligent system that can filter through vast book collections and recommend titles tailored to individual user preferences

## Objectives

### General Objective
To develop a machine learning-based book recommender system that provides personalized book suggestions to users based on past ratings and book attributes.

### Specific Objectives
- To analyze and clean the book and rating datasets for accurate modeling.
- To implement **collaborative filtering** techniques (user-based and item-based) using historical rating data.
- To build a **content-based filtering** model that uses book metadata (e.g., title, author, genre).
- To evaluate the performance of the recommender models using relevant metrics such as RMSE, precision, and recall.
- To visualize trends in user preferences and book popularity.


In [21]:
pip install --upgrade pandas


   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
    --------------------------------------- 0.3/10.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/10.8 MB 1.3 MB/s eta 0:00:08
   - -------------------------------------- 0.5/10.8 MB 1.3 MB/s eta 0:00:08
   --- ------------------------------------ 1.0/10.8 MB 1.1 MB/s eta 0:00:09
   --- ------------------------------------ 1.0/10.8 MB 1.1 MB/s eta 0:00:09
   ---- ----------------------------------- 1.3/10.8 MB 1.0 MB/s eta 0:00:10
   ----- ---------------------------------- 1.6/10.8 MB 1.0 MB/s eta 0:00:09
   ------ --------------------------------- 1.8/10.8 MB 1.0 MB/s eta 0:00:09
   ------ --------------------------------- 1.8/10.8 MB 1.0 MB/s eta 0:00:09
   ------ --------------------------------- 1.8/10.8 MB 1.0 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/10.8 MB 821.0 kB/s eta 0:00:11
   ------- -------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fbprophet 0.7.1 requires cmdstanpy==0.9.5, which is not installed.


In [22]:
# Loading the data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
books_df = pd.read_csv(
    r'D:\PROJECT\Online-Book-Recommender-System\books_df.csv',
    sep=';',
    quotechar='"',
    encoding='latin1',
    error_bad_lines=False  # use this instead of on_bad_lines for older pandas
)

# Display the first 10 rows
books_df.head(10)


b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
c:\Users\LENOVO\anaconda3\envs\learn-env\lib\site-packages\IPython\core\in

AttributeError: 'NotebookFormatter' object has no attribute 'get_result'

         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   
5  0399135782                             The Kitchen God's Wife   
6  0425176428  What If?: The World's Foremost Military Histor...   
7  0671870432                                    PLEADING GUILTY   
8  0679425608  Under the Black Flag: The Romance and the Real...   
9  074322678X            Where You'll Find Me: And Other Stories   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este   